## Import

In [1]:
import random
import pandas as pd
import numpy as np
import os
import re
import glob
import math
import random
from PIL import Image
from util import *

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

from transformers import SwinForImageClassification, AutoImageProcessor, Trainer, TrainingArguments

import wandb

from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm

import warnings
warnings.filterwarnings(action='ignore') 

## Define Environment Variables

In [2]:
%env WANDB_PROJECT=WallPaperDefectTypeClassification
%env WANDB_NOTEBOOK_NAME=./experiment.ipynb
%env WANDB_LOG_MODEL=end
%env WANDB_WATCH=all
%env WANDB_RUN_GROUP=exp0
%env WANDB_JOB_TYPE=train

env: WANDB_PROJECT=WallPaperDefectTypeClassification
env: WANDB_NOTEBOOK_NAME=./experiment.ipynb
env: WANDB_LOG_MODEL=end
env: WANDB_WATCH=all
env: WANDB_RUN_GROUP=exp0
env: WANDB_JOB_TYPE=train


In [3]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device.type

'cuda'

## Hyperparameter Setting

In [4]:
CFG = {
    'IMG_SIZE':224,
    'EPOCHS':10,
    'LEARNING_RATE':3e-4,
    'BATCH_SIZE':8,
    'WEIGHT_DECAY':0.01,
    'WARMUP_RATIO':0.1,
    'SEED':42,
    'NUM_WORKERS':4,
    'PRETRAINED_MODEL': "microsoft/swin-base-patch4-window7-224-in22k",
    'MODEL_VER' : "0.0.3_tiny+default A",
}

## Fixed RandomSeed

In [5]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

## Data Pre-processing

In [6]:
all_img_list = glob.glob('../data/train/*/*')

In [7]:
df = pd.DataFrame(columns=['img_path', 'label'])
df['img_path'] = all_img_list
df['label'] = df['img_path'].apply(lambda x: str(x).split('/')[-2]).astype(int)

In [8]:
train, val, _, _ = train_test_split(df, df['label'], test_size=0.3, stratify=df['label'], random_state=CFG['SEED'])

## CustomDataset

<img src=https://d2.naver.com/content/images/2021/01/efbe9400-5214-11eb-9c67-30fab62770ec.png>

**Albumentation Tutorials**<br>
https://github.com/albumentations-team/albumentations_examples/blob/master/notebooks/migrating_from_torchvision_to_albumentations.ipynb

In [9]:
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, transforms=None, processor=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms
        self.processor = processor

    def __getitem__(self, index):
        
        img_path = self.img_path_list[index]
        image = Image.open(img_path)
        image_tr = self.transforms(image=np.array(image))['image']
        pixel_values = self.processor(image_tr, return_tensors="pt").pixel_values.squeeze()
        
        if self.label_list is not None:
            label = self.label_list[index]
            return {
                'pixel_values': pixel_values, 
                'label': label,
                }
        else:
            return {
                'pixel_values': pixel_values,
                }
        
    def __len__(self):
        return len(self.img_path_list)

In [10]:
train_transform = A.Compose([
                            A.HorizontalFlip(p=0.5),
                            A.RandomBrightnessContrast(p=0.5),
                            A.RandomScale(scale_limit=0.1, p=0.5),
                            A.RandomCropFromBorders(p=0.5),
                            ToTensorV2()
                            ])

test_transform = A.Compose([
                            # Augmentations
                            ToTensorV2()
                            ])

In [11]:
model_checkpoint = CFG['PRETRAINED_MODEL']
image_processor = AutoImageProcessor.from_pretrained(model_checkpoint)

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


In [12]:
train_dataset = CustomDataset(
    train['img_path'].values, train['label'].values, train_transform, image_processor)

val_dataset = CustomDataset(
    val['img_path'].values, val['label'].values, test_transform, image_processor)


## Model Define

In [13]:
def model_init():
    labels = pd.read_csv("../data/map.csv")['Categories']
    model = SwinForImageClassification.from_pretrained(
        model_checkpoint,
        num_labels=len(labels),
        id2label={str(i): c for i, c in enumerate(labels)},
        label2id={c: str(i) for i, c in enumerate(labels)},
        ignore_mismatched_sizes=True,
    ).to(device)
    return model


## Train

In [14]:
import evaluate

metric = evaluate.load("f1")

def compute_metrics(p):
  return metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids, average='weighted')


In [15]:
def collate_fn(batch):
    return {
        'pixel_values': torch.stack([x['pixel_values'] for x in batch]),
        'labels': torch.tensor([x['label'] for x in batch]).type(torch.LongTensor),
    }

In [16]:
sweep_config = {
    'method': 'random',
    'metric' : {
        'name': 'eval/f1',
        'goal': 'maximize'   
        },
    'parameters' : {
        'learning_rate': {
            'distribution': 'log_uniform_values',
            'min': 1e-5,
            'max': 1e-3
        },
        'weight_decay': {
            'distribution': 'q_uniform',
            'min': 0,
            'max': 1e-2,
            'q': 0.001
        },
        'warmup_ratio':{
            'values': [0.0, 0.1, 0.2]
        },
        'lr_scheduler_type':{
            'values': ['linear', 'cosine']
        },
        'batch_size':{
            'values': 32
        }
    },
    'early_terminate': {
        'type': 'hyperband',
        'min_iter': 3,
        'eta' : 2
    },
}

In [17]:
sweep_id = wandb.sweep(sweep_config, project=os.environ['WANDB_PROJECT'])

Create sweep with ID: cftsawev
Sweep URL: https://wandb.ai/2gnldud/WallPaperDefectTypeClassification/sweeps/cftsawev


```
class SchedulerType(ExplicitEnum):
    LINEAR = "linear"
    COSINE = "cosine"
    COSINE_WITH_RESTARTS = "cosine_with_restarts"
    POLYNOMIAL = "polynomial"
    CONSTANT = "constant"
    CONSTANT_WITH_WARMUP = "constant_with_warmup"
    INVERSE_SQRT = "inverse_sqrt"
```


In [18]:
model_name = model_checkpoint.split("/")[-1]

def train(config=None):
    with wandb.init(config=config):
        config = wandb.config
        args = TrainingArguments(
            output_dir=f"../outputs/{model_name}-finetuned",
            overwrite_output_dir=True,
            remove_unused_columns=False,
            evaluation_strategy="epoch",
            save_strategy="epoch",
            learning_rate=config.learning_rate,
            per_device_train_batch_size=config.batch_size,
            per_device_eval_batch_size=config.batch_size,
            num_train_epochs=CFG['EPOCHS'],
            weight_decay=config.weight_decay,
            logging_steps=10,
            gradient_accumulation_steps=4,
            dataloader_num_workers=CFG['NUM_WORKERS'],
            warmup_ratio=config.warmup_ratio,
            fp16=True,
            lr_scheduler_type=config.lr_scheduler_type,
            load_best_model_at_end=True,
            metric_for_best_model="f1",
            run_name='v.'+CFG['MODEL_VER'],
            report_to='wandb',
        )

        trainer = Trainer(
            model_init=model_init,
            args=args,
            data_collator=collate_fn,
            compute_metrics=compute_metrics,
            train_dataset=train_dataset,
            eval_dataset=val_dataset,
            tokenizer=image_processor,
        )

        trainer.train()

In [19]:
wandb.agent(sweep_id, train, count=16)
wandb.finish()

wandb: Agent Starting Run: em7zogky with config:
wandb: 	batch_size: 32
wandb: 	learning_rate: 0.0002738450494723399
wandb: 	lr_scheduler_type: constant_with_warmup
wandb: 	warmup_ratio: 0.1
wandb: 	weight_decay: 0.002
wandb: Currently logged in as: 2gnldud. Use `wandb login --relogin` to force relogin


Some weights of SwinForImageClassification were not initialized from the model checkpoint at microsoft/swin-base-patch4-window7-224-in22k and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([21841, 1024]) in the checkpoint and torch.Size([19, 1024]) in the model instantiated
- classifier.bias: found shape torch.Size([21841]) in the checkpoint and torch.Size([19]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of SwinForImageClassification were not initialized from the model checkpoint at microsoft/swin-base-patch4-window7-224-in22k and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([21841, 1024]) in the checkpoint and torch.Size([19, 1024]) in the model instantiated
- classifier.bias: found shape torch.Size([21841]) in the checkpoint and torch.Size([19]) in the model instantiated
Y

Epoch,Training Loss,Validation Loss,F1
1,2.283100,0.926181,0.717421
2,0.757500,0.610237,0.797939
3,0.395800,0.494081,0.846583
4,0.247400,0.700539,0.809256


eval/f1,▁▅█▆▆▇▇█▆▇
eval/loss,█▃▁▄▄▄▄▃▆▆
eval/runtime,▂▅▆█▄▁▃▃▃▃
eval/samples_per_second,▆▃▂▁▄█▅▅▅▅
eval/steps_per_second,▆▃▂▁▄█▅▅▅▅
train/epoch,▁▁▁▂▂▂▃▃▃▃▄▄▄▄▄▅▅▅▆▆▆▆▇▇▇▇████
train/global_step,▁▁▁▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇████
train/learning_rate,▁██████████████████
train/loss,█▅▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁
train/total_flos,▁
train/train_loss,▁


wandb: Agent Starting Run: bnnymapk with config:
wandb: 	batch_size: 16
wandb: 	learning_rate: 4.944008967957393e-05
wandb: 	lr_scheduler_type: constant_with_warmup
wandb: 	warmup_ratio: 0
wandb: 	weight_decay: 0.009000000000000001


...

wandb: Agent Starting Run: 9vs9l3uv with config:
wandb: 	batch_size: 32
wandb: 	learning_rate: 1.628883109956865e-05
wandb: 	lr_scheduler_type: linear
wandb: 	warmup_ratio: 0
wandb: 	weight_decay: 0.004


Some weights of SwinForImageClassification were not initialized from the model checkpoint at microsoft/swin-base-patch4-window7-224-in22k and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([21841, 1024]) in the checkpoint and torch.Size([19, 1024]) in the model instantiated
- classifier.bias: found shape torch.Size([21841]) in the checkpoint and torch.Size([19]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of SwinForImageClassification were not initialized from the model checkpoint at microsoft/swin-base-patch4-window7-224-in22k and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([21841, 1024]) in the checkpoint and torch.Size([19, 1024]) in the model instantiated
- classifier.bias: found shape torch.Size([21841]) in the checkpoint and torch.Size([19]) in the model instantiated
Y

Epoch,Training Loss,Validation Loss,F1
1,2.423300,1.552098,0.458652


wandb: Ctrl + C detected. Stopping sweep.


eval/f1,▁
eval/loss,▁
eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
train/epoch,▁█
train/global_step,▁█
train/learning_rate,▁
train/loss,▁
eval/f1,0.45865
eval/loss,1.5521
